# Example of cryptostatemachine
Simulate P2P thing-lending with microinsurance with Python module *cryptostatemachine*

## Characters
Alice: Has digital camera she rarely uses

Bob: Wants digital camera for family bbq next weekend

AZ: Offers novel, exciting microinsurance solutions on the blockchain

## Data structures
*StateMachine*: a directed-graph based Python class implementation of a state-machine with interactions encoded in both the graph structure and ornamentation. As state evolves, crytpographic Bitcoin-like transaction data is exported to a database (currently json file, next step: blockchain)

*SimpleSM*: descendant of StateMachine, with fixed graph structure for simplest P2P thing-lending work flows.

In [118]:
import os
import sys
import ecdsa as ec #Pure Python elliptic curve cryptography
import binascii as ba

In [119]:
# Attach local path where 
module_path = os.path.abspath(os.path.join('PycharmProjects/csm'))

if module_path not in sys.path:
    sys.path.append(module_path)


In [120]:
import cryptostatemachine.cryptostatemachine as csm
reload(csm)

<module 'cryptostatemachine.cryptostatemachine' from '/home/pavel/PycharmProjects/csm/cryptostatemachine/cryptostatemachine.pyc'>

Define the characters

In [121]:
# Issuer
alice_priv = ec.SigningKey.generate()
alice_priv_hex = ba.b2a_hex(alice_priv.to_string())
alice_pub = alice_priv.get_verifying_key()
alice_pub_hex = ba.b2a_hex(alice_pub.to_string())
print 'Alice\'s public key:', alice_pub_hex, '\n'

# Interested party
bob_priv = ec.SigningKey.generate()
bob_priv_hex = ba.b2a_hex(bob_priv.to_string())
bob_pub = bob_priv.get_verifying_key()
bob_pub_hex = ba.b2a_hex(bob_pub.to_string())
print 'Bob\'s public key:', bob_pub_hex, '\n'

# Microinsurer
az_priv = ec.SigningKey.generate()
az_priv_hex = ba.b2a_hex(az_priv.to_string())
az_pub = az_priv.get_verifying_key()
az_pub_hex = ba.b2a_hex(az_pub.to_string())
print 'AZ public key:', az_pub_hex, '\n'

# P2P platform
p2p_priv = ec.SigningKey.generate()
p2p_priv_hex = ba.b2a_hex(p2p_priv.to_string())
p2p_pub = p2p_priv.get_verifying_key()
p2p_pub_hex = ba.b2a_hex(p2p_pub.to_string())
#print 'P2P public key:', p2p_pub

# JSON file to hold transaction data for blockchain (mock-up)
db_file = 'p2pthing.json' # Output file for transaction data


Alice's public key: 48f31de94f8c7320e1a6d29c26f3c214e662aed6dc89d59c991779b21154338efcfc1b51ad2066aefff410044fec6fc4 

Bob's public key: ab7936d0d34f780e82c636dd8d3ea8b984ec3477f5f9b0f31dfb90e42f46ce5536dea6ea4da472885ad0cb6b38b65335 

AZ public key: 3c9cb031fac68145079e612d99a45e7fa3c0ad9939218ba370de2161b12571aa7b68518573e16b7992d8822b3bd798bf 



Let's look at what is inside an instance of a simple csm.

In [122]:
# Post offer of camera on P2P platform, back-end creates instance of SimpleSM
p2p_message = 'It looks like Alice wants to post a camera on our platform'
p2p_msg_signed = ba.b2a_hex(p2p_priv.sign(p2p_message))
ssm = csm.SimpleSM(p2p_pub_hex, p2p_msg_signed) # Initialize, and track which p2p platform is calling
# Initial graph

print 'Initial Graph Nodes: \n', ssm.graph.nodes(), '\n'

print 'Initial Graph Edges: \n',ssm.graph.edges(), '\n'

# Decoration of vertices:
print 'Initial Node Decoration: \n', ssm.graph['initial'], '\n'

# Hashing
print 'Instance time stame: \n', ssm.time_stamp, '\n'
print 'Initial instance digest: \n', ssm.whole_digest, '\n'


Initial Graph Nodes: 
['returned', 'offered', 'initial', 'transferred', 'insured', 'interested'] 

Initial Graph Edges: 
[('offered', 'insured'), ('offered', 'interested'), ('initial', 'offered'), ('initial', 'statehashdigest'), ('initial', 'sender'), ('initial', 'signature'), ('transferred', 'returned'), ('interested', 'transferred')] 

Initial Node Decoration: 
{'offered': {}, 'statehashdigest': '3a183b2d42d6d015f6bd8b4ba8cc1510b8a8591cf2227db06e567002bd56cc3e', 'sender': 'faacca4841027c3e00feba6e08fbc03af6915f23b6d10f46672f38f016884ed4b548432799610d599a6f0fffdb806950', 'signature': 'd46b324869233f5a1e7a2db89be0b531cd9b05365e985c132f8fa467014ad6a1fd2f11d5c43df92c66c28abf41cb06a6'} 

Instance time stame: 
86a3a76f5b3918608d62e37826e1c239678efbc18113136cca4bf0016c0d84b3 

Initial instance digest: 
3a183b2d42d6d015f6bd8b4ba8cc1510b8a8591cf2227db06e567002bd56cc3e 



And here are the initial transaction data. Note that the "in_data" are initialized to nonsense.

In [123]:
# Initial transaction (p2p platform only)
print(ssm.trans_data.get_as_dict())
ssm.export_trans_data(db_file)

{'meta_data': {'ver': -1, 'lock_time': -1, 'vin_sz': -1, 'hash': '3a183b2d42d6d015f6bd8b4ba8cc1510b8a8591cf2227db06e567002bd56cc3e', 'size': -1}, 'out_data': {'scrip_pub_key': -1, 'value': -1}, 'in_data': {'hash': -1, 'n': -1}}


In [124]:
# Alice offers camera on p2p platform
alice_message = 'Anyone interested in my camera? It\'s a Sony RX100 II'
alice_msg_signed = ba.b2a_hex(alice_priv.sign(alice_message))
# Add decoration to 'offered' node
ssm.update_node(node_name = 'offered',
                  sender = alice_pub_hex,
               signature = alice_msg_signed)

print 'Graph structure as string for hashing: \n', ssm.get_graph_str(), '\n'
print 'Instance digest after update: \n', ssm.whole_digest
# Look at transaction data after node update
print 'Transaction data after camera offer placed: \n', ssm.trans_data.get_as_dict()
#Export to data base
ssm.export_trans_data(db_file)


Graph structure as string for hashing: 
{'returned': {}, 'offered': {'insured': {}, 'interested': {}, 'statehashdigest': '8c14d05bb39d13c89f812fbb623076c34b3a675fff590deb61502d3dc6c7bcb6', 'sender': '48f31de94f8c7320e1a6d29c26f3c214e662aed6dc89d59c991779b21154338efcfc1b51ad2066aefff410044fec6fc4', 'signature': '01bbd1e9d40e05f0050949447b4bc9b4027ae7a6abb98d6089e81e6a5ace9a06466bc49e3838a67e4fb25d68351e28ca'}, 'initial': {'offered': {}, 'statehashdigest': '3a183b2d42d6d015f6bd8b4ba8cc1510b8a8591cf2227db06e567002bd56cc3e', 'sender': 'faacca4841027c3e00feba6e08fbc03af6915f23b6d10f46672f38f016884ed4b548432799610d599a6f0fffdb806950', 'signature': 'd46b324869233f5a1e7a2db89be0b531cd9b05365e985c132f8fa467014ad6a1fd2f11d5c43df92c66c28abf41cb06a6'}, 'transferred': {'returned': {}}, 'insured': {}, 'interested': {'transferred': {}}} 

Instance digest after update: 
8c14d05bb39d13c89f812fbb623076c34b3a675fff590deb61502d3dc6c7bcb6
Transaction data after camera offer placed: 
{'meta_data': {'ver': -

In [125]:
# Bob sees the post and clicks to ask about the camera
bob_message = 'Ohh, a Sony RX100 II. I would like to borrow it.'
bob_msg_signed = ba.b2a_hex(bob_priv.sign(bob_message))
# Add decoration to 'offered' node
ssm.update_node(node_name = 'interested',
                  sender = bob_pub_hex,
               signature = bob_msg_signed)

print 'Graph structure as string for hashing: \n', ssm.get_graph_str(), '\n'
print 'Instance digest after update: \n', ssm.whole_digest
# Look at transaction data after node update
print 'Transaction data after camera offer placed: \n', ssm.trans_data.get_as_dict()
#Export to data base
ssm.export_trans_data(db_file)


Graph structure as string for hashing: 
{'returned': {}, 'offered': {'insured': {}, 'interested': {}, 'statehashdigest': '8c14d05bb39d13c89f812fbb623076c34b3a675fff590deb61502d3dc6c7bcb6', 'sender': '48f31de94f8c7320e1a6d29c26f3c214e662aed6dc89d59c991779b21154338efcfc1b51ad2066aefff410044fec6fc4', 'signature': '01bbd1e9d40e05f0050949447b4bc9b4027ae7a6abb98d6089e81e6a5ace9a06466bc49e3838a67e4fb25d68351e28ca'}, 'initial': {'offered': {}, 'statehashdigest': '3a183b2d42d6d015f6bd8b4ba8cc1510b8a8591cf2227db06e567002bd56cc3e', 'sender': 'faacca4841027c3e00feba6e08fbc03af6915f23b6d10f46672f38f016884ed4b548432799610d599a6f0fffdb806950', 'signature': 'd46b324869233f5a1e7a2db89be0b531cd9b05365e985c132f8fa467014ad6a1fd2f11d5c43df92c66c28abf41cb06a6'}, 'transferred': {'returned': {}}, 'insured': {}, 'interested': {'sender': 'ab7936d0d34f780e82c636dd8d3ea8b984ec3477f5f9b0f31dfb90e42f46ce5536dea6ea4da472885ad0cb6b38b65335', 'statehashdigest': '299b2841f40b7968ce47df4a3108629497680b2373ff955930d1fbcf

In [126]:
# Alice sees that Bob is interested, and applies for microinsurance with AZ

az_message = 'We can insure your Sony RX100 II for xyz under conditions abc'
az_msg_signed = az_priv.sign(az_message)
az_signed_hex = ba.b2a_hex(az_msg_signed)
# Add decoration to 'offered' node
ssm.update_node(node_name = 'insured',
                  sender = az_pub_hex,
               signature = az_signed_hex)

print 'Graph structure as string for hashing: \n', ssm.get_graph_str(), '\n'
print 'Instance digest after update: \n', ssm.whole_digest
# Look at transaction data after node update
print 'Transaction data after camera offer placed: \n', ssm.trans_data.get_as_dict()
#Export to data base
ssm.export_trans_data(db_file)

Graph structure as string for hashing: 
{'returned': {}, 'offered': {'insured': {}, 'interested': {}, 'statehashdigest': '8c14d05bb39d13c89f812fbb623076c34b3a675fff590deb61502d3dc6c7bcb6', 'sender': '48f31de94f8c7320e1a6d29c26f3c214e662aed6dc89d59c991779b21154338efcfc1b51ad2066aefff410044fec6fc4', 'signature': '01bbd1e9d40e05f0050949447b4bc9b4027ae7a6abb98d6089e81e6a5ace9a06466bc49e3838a67e4fb25d68351e28ca'}, 'initial': {'offered': {}, 'statehashdigest': '3a183b2d42d6d015f6bd8b4ba8cc1510b8a8591cf2227db06e567002bd56cc3e', 'sender': 'faacca4841027c3e00feba6e08fbc03af6915f23b6d10f46672f38f016884ed4b548432799610d599a6f0fffdb806950', 'signature': 'd46b324869233f5a1e7a2db89be0b531cd9b05365e985c132f8fa467014ad6a1fd2f11d5c43df92c66c28abf41cb06a6'}, 'transferred': {'returned': {}}, 'insured': {'statehashdigest': '44e193a1c889ee9f958e7b8382fa32c97843b62252aace10b11bc4e8654f798b', 'sender': '3c9cb031fac68145079e612d99a45e7fa3c0ad9939218ba370de2161b12571aa7b68518573e16b7992d8822b3bd798bf', 'signat

In [127]:
# Alice checks that insurance has been offered,
# and verifies that it is AZ that signed the offer

ssm.graph['insured']
check_az_key = ssm.graph['insured']['sender']
print check_az_key == az_pub_hex

print az_pub.verify(az_msg_signed, az_message) #TODO put instances in graph, not just hexes


True
True


In [128]:
# Alice ships the camera to Bob
shipped_message = 'The camera has been shipped via DHL nr 31415926535'
alice_ship_signed = ba.b2a_hex(alice_priv.sign(shipped_message))
# Add decoration to 'offered' node
ssm.update_node(node_name = 'transferred',
                  sender = alice_pub_hex,
               signature = alice_ship_signed)

print 'Graph structure as string for hashing: \n', ssm.get_graph_str(), '\n'
print 'Instance digest after update: \n', ssm.whole_digest
# Look at transaction data after node update
print 'Transaction data after camera offer placed: \n', ssm.trans_data.get_as_dict()
#Export to data base
ssm.export_trans_data(db_file)

Graph structure as string for hashing: 
{'returned': {}, 'offered': {'insured': {}, 'interested': {}, 'statehashdigest': '8c14d05bb39d13c89f812fbb623076c34b3a675fff590deb61502d3dc6c7bcb6', 'sender': '48f31de94f8c7320e1a6d29c26f3c214e662aed6dc89d59c991779b21154338efcfc1b51ad2066aefff410044fec6fc4', 'signature': '01bbd1e9d40e05f0050949447b4bc9b4027ae7a6abb98d6089e81e6a5ace9a06466bc49e3838a67e4fb25d68351e28ca'}, 'initial': {'offered': {}, 'statehashdigest': '3a183b2d42d6d015f6bd8b4ba8cc1510b8a8591cf2227db06e567002bd56cc3e', 'sender': 'faacca4841027c3e00feba6e08fbc03af6915f23b6d10f46672f38f016884ed4b548432799610d599a6f0fffdb806950', 'signature': 'd46b324869233f5a1e7a2db89be0b531cd9b05365e985c132f8fa467014ad6a1fd2f11d5c43df92c66c28abf41cb06a6'}, 'transferred': {'sender': '48f31de94f8c7320e1a6d29c26f3c214e662aed6dc89d59c991779b21154338efcfc1b51ad2066aefff410044fec6fc4', 'statehashdigest': '0929692238ecd92630cbd7ee7260e557b04534ebc52fa77b107b681442e78250', 'returned': {}, 'signature': 'a2494e

In [129]:
# Happy Ending
# Bob returns the camera to Alice
returned_message = 'The camera was great, shipped back via DHL nr 271828'
bob_ship_signed = ba.b2a_hex(bob_priv.sign(returned_message))
# Add decoration to 'offered' node
ssm.update_node(node_name = 'returned',
                  sender = bob_pub_hex,
               signature = bob_ship_signed)

print 'Graph structure as string for hashing: \n', ssm.get_graph_str(), '\n'
print 'Instance digest after update: \n', ssm.whole_digest
# Look at transaction data after node update
print 'Transaction data after camera offer placed: \n', ssm.trans_data.get_as_dict()
#Export to data base
ssm.export_trans_data(db_file)

Graph structure as string for hashing: 
{'returned': {'statehashdigest': '6d0209a2002a8e37bed7f672971835599634b4380b74aa1494c82fb2b85518bc', 'sender': 'ab7936d0d34f780e82c636dd8d3ea8b984ec3477f5f9b0f31dfb90e42f46ce5536dea6ea4da472885ad0cb6b38b65335', 'signature': 'b7fc75c43a766b71dc3f73f68cd9a928dddf1279a672ae65b87cd81e855159c7a7ac1bd1df6afd66567a09eebb101618'}, 'offered': {'insured': {}, 'interested': {}, 'statehashdigest': '8c14d05bb39d13c89f812fbb623076c34b3a675fff590deb61502d3dc6c7bcb6', 'sender': '48f31de94f8c7320e1a6d29c26f3c214e662aed6dc89d59c991779b21154338efcfc1b51ad2066aefff410044fec6fc4', 'signature': '01bbd1e9d40e05f0050949447b4bc9b4027ae7a6abb98d6089e81e6a5ace9a06466bc49e3838a67e4fb25d68351e28ca'}, 'initial': {'offered': {}, 'statehashdigest': '3a183b2d42d6d015f6bd8b4ba8cc1510b8a8591cf2227db06e567002bd56cc3e', 'sender': 'faacca4841027c3e00feba6e08fbc03af6915f23b6d10f46672f38f016884ed4b548432799610d599a6f0fffdb806950', 'signature': 'd46b324869233f5a1e7a2db89be0b531cd9b05365

In [130]:
# Alternative ending
# Bob doesn't return camera, AZ pays out claim (coming soon . . .)